In [ ]:
import pandas as pd
from sklearn import preprocessing
from collections import deque
import random
import numpy as np

In [ ]:
df=pd.read_csv('LTC-USD.csv',names=["time","low","high","open","close","volume"])
df.head()

,time,low,high,open,close,volume
0,1528968660,96.580002,96.589996,96.589996,96.580002,9.647200
1,1528968720,96.449997,96.669998,96.589996,96.660004,314.387024
2,1528968780,96.470001,96.570000,96.570000,96.570000,77.129799
3,1528968840,96.449997,96.570000,96.570000,96.500000,7.216067
4,1528968900,96.279999,96.540001,96.500000,96.389999,524.539978


In [ ]:
print(df.shape)

(101883, 6)


In [ ]:
SEQ_LEN=60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT="LTC-USD"

In [ ]:
main_df=pd.DataFrame()

In [ ]:
ratios=["BCH-USD","LTC-USD","ETH-USD","BTC-USD"]
for ratio in ratios:
    dataset=f"{ratio}.csv"
    df=pd.read_csv(dataset,names=["time","low","high","open","close","volume"])
    df.rename(columns={"close":f"{ratio}_close","volume":f"{ratio}_volume"},inplace=True)
    df.set_index("time",inplace=True)
    df=df[[f"{ratio}_close",f"{ratio}_volume"]]
    if len(main_df)==0:
      main_df=df
    else:
      main_df=main_df.join(df)

In [ ]:
print(main_df.head())

            BCH-USD_close  BCH-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528968660     871.719971        5.675361      96.580002        9.647200   
1528968720     870.859985       26.856577      96.660004      314.387024   
1528968780     870.099976        1.124300      96.570000       77.129799   
1528968840     870.789978        1.749862      96.500000        7.216067   
1528968900     870.000000        1.680500      96.389999      524.539978   

            ETH-USD_close  ETH-USD_volume  BTC-USD_close  BTC-USD_volume  
time                                                                      
1528968660            NaN             NaN    6489.549805        0.587100  
1528968720      486.01001       26.019083    6487.379883        7.706374  
1528968780      486.00000        8.449400    6479.410156        3.088252  
1528968840      485.75000       26.994646    6479.410156        1.404100  
1528968900      4

In [ ]:
main_df['future']=main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

In [ ]:
print(main_df[['future',f"{RATIO_TO_PREDICT}_close"]])

               future  LTC-USD_close
time                                
1528968660  96.500000      96.580002
1528968720  96.389999      96.660004
1528968780  96.519997      96.570000
1528968840  96.440002      96.500000
1528968900  96.470001      96.389999
...               ...            ...
1535215020  58.080002      58.009998
1535215080  58.090000      58.020000
1535215140        NaN      58.020000
1535215200        NaN      58.080002
1535215260        NaN      58.090000

[92225 rows x 2 columns]


In [ ]:
def classify(current,future):
  if future>current:
    return 1
  else:
    return 0

In [ ]:
def preproccess_df(df):
  df.drop('future',axis=1,inplace=True)
  for col in df.columns:
    if col != "target":
      df[col]=df[col].pct_change()
      df.dropna(inplace=True)
      df[col]=preprocessing.scale(df[col].values)
  df.dropna(inplace=True)
  sequential_data=[]
  prev_days= deque(maxlen=SEQ_LEN)

  for i in df.values:
    prev_days.appen([n for n in i[:-1]])

In [ ]:
main_df['target']=list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df['future']))

In [ ]:
print(main_df[['future',f"{RATIO_TO_PREDICT}_close",'target']].head())

               future  LTC-USD_close  target
time                                        
1528968660  96.500000      96.580002       0
1528968720  96.389999      96.660004       0
1528968780  96.519997      96.570000       0
1528968840  96.440002      96.500000       0
1528968900  96.470001      96.389999       1


In [ ]:
times = sorted(main_df.index.values)

In [ ]:
last_5pc=times[-int(0.05*len(times))]

In [ ]:
validation_main_df=main_df[(main_df.index)>=last_5pc]

In [ ]:
main_df=main_df[(main_df.index<last_5pc)]

In [ ]:
preproccess_df(main_df)

            BCH-USD_close  BCH-USD_volume  LTC-USD_close  LTC-USD_volume  \
time                                                                       
1528969140       0.010836       -0.006232      -0.541363       -0.117375   
1528969200       0.272971       -0.005900       0.004432       -0.118601   
1528969260       0.281333       -0.006215       0.004432       -0.072976   
1528969320       0.483962       -0.005915       0.004432       -0.108465   
1528969380       0.559719       -0.006221       0.004432       -0.048325   

            ETH-USD_close  ETH-USD_volume  BTC-USD_close  BTC-USD_volume  \
time                                                                       
1528969140       0.004951       -0.047803      -0.002086       -0.078763   
1528969200       0.004951       -0.045352      -0.229580        0.003264   
1528969260       0.021689       -0.032924      -0.000403       -0.073901   
1528969320       0.004951       -0.049489       0.116733       -0.049578   
1528969380 